# --- 00_CONFIGURAÇÕES E IMPORTS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu, spearmanr
!pip install yfinance
import yfinance as yf


#01_ CARREGAMENTO DOS DADOS

In [2]:
from google.colab import drive
drive.mount('/content/drive')

caminho_base = '/content/drive/My Drive/INDICIUM_DESAFIO/01_data/'

df_agencias = pd.read_csv(caminho_base + 'agencias.csv')
df_clientes = pd.read_csv(caminho_base + 'clientes.csv')
df_colaborador_agencia = pd.read_csv(caminho_base + 'colaborador_agencia.csv')
df_colaboradores = pd.read_csv(caminho_base + 'colaboradores.csv')
df_contas = pd.read_csv(caminho_base + 'contas.csv')
df_propostas_credito = pd.read_csv(caminho_base + 'propostas_credito.csv')
df_transacoes = pd.read_csv(caminho_base + 'transacoes.csv')



Mounted at /content/drive


# 02_ETL

In [3]:

df_clientes['data_inclusao'] = pd.to_datetime(df_clientes['data_inclusao'])
df_clientes['data_nascimento'] = pd.to_datetime(df_clientes['data_nascimento'])
df_contas['data_abertura'] = pd.to_datetime(df_contas['data_abertura'])
df_contas['data_ultimo_lancamento'] = pd.to_datetime(df_contas['data_ultimo_lancamento'], format='mixed', utc=True)
df_transacoes['data_transacao'] = pd.to_datetime(df_transacoes['data_transacao'], format='mixed', utc=True)

 # 03_MODELAGEM E UNIFICAÇÃO

In [5]:
transacoes_com_contas = pd.merge(df_transacoes, df_contas, on='num_conta', how='left')
transacoes_com_clientes = pd.merge(transacoes_com_contas, df_clientes, on='cod_cliente', how='left')
df_final = pd.merge(transacoes_com_clientes, df_agencias, on='cod_agencia', how='left')
df_final = df_final.rename(columns={
    'data_abertura_x': 'data_abertura_conta', 'endereco_x': 'endereco_cliente', 'nome': 'nome_agencia',
    'endereco_y': 'endereco_agencia', 'data_abertura_y': 'data_abertura_agencia'
})
# Criando colunas de data úteis no df_final
df_final['year_month'] = df_final['data_transacao'].dt.strftime('%Y-%m')
df_final['month'] = df_final['data_transacao'].dt.month

#04_ANÁLISE TEMPORAL
*texto em itálico*

In [10]:

#04_01_VOLUME_POR_MES
volume_mensal = df_final.groupby('year_month').agg(
    valor_total_mes=('valor_transacao', 'sum'),
    mes_numero=('month', 'first')
).reset_index()

#04_02_MES_PAR_IMPAR
volume_mensal['tipo_mes'] = np.where(volume_mensal['mes_numero'] % 2 == 0, 'Par', 'Ímpar')

#04_03_MÉDIA_PAR_IMPAR
analise_meses = volume_mensal.groupby('tipo_mes').agg(
    media_mensal_valor=('valor_total_mes', 'mean')
).reset_index()

#05_ANÁLISE_DOL


In [11]:
#05_01_DADOS_COT_DOLAR
start_date_yf = df_final['data_transacao'].min().strftime('%Y-%m-%d')
end_date_yf = df_final['data_transacao'].max().strftime('%Y-%m-%d')
dolar_data = yf.download('BRL=X', start=start_date_yf, end=end_date_yf, progress=False)
dolar_data.columns = dolar_data.columns.get_level_values(0)
dolar_daily = dolar_data[['Close']].reset_index().rename(columns={'Date': 'data_dia', 'Close': 'cotacao_dolar'})
dolar_daily['data_dia'] = pd.to_datetime(dolar_daily['data_dia'])
#05_02_PREP_DADOS
df_final['data_dia'] = pd.to_datetime(df_final['data_transacao'].dt.date)
transacoes_diarias = df_final.groupby('data_dia').agg(
    numero_de_transacoes=('cod_transacao', 'count')
).reset_index()
#05_03_JOIN_TAB
analise_dolar = pd.merge(transacoes_diarias, dolar_daily, on='data_dia', how='left').dropna()

/tmp/ipython-input-21373725.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dolar_data = yf.download('BRL=X', start=start_date_yf, end=end_date_yf, progress=False)


#06_ANALISE_AGENC

In [16]:
max_date = df_final['data_transacao'].max()
start_6m = max_date - pd.DateOffset(months=6)
df_6m = df_final[df_final['data_transacao'] >= start_6m]
rank_ag = df_6m.groupby('nome_agencia').agg(
    n_transacoes=('cod_transacao','count'),
    valor_total=('valor_transacao','sum'),
    ticket_medio=('valor_transacao','mean')
).sort_values('n_transacoes', ascending=False).reset_index()

print("Top 3 agências (últimos 6 meses):")
print(rank_ag.head(3))
print("\nBottom 3 agências (últimos 6 meses):")
print(rank_ag.tail(3))

Top 3 agências (últimos 6 meses):
      nome_agencia  n_transacoes  valor_total  ticket_medio
0  Agência Digital         16806   4647035.10    276.510478
1   Agência Matriz          2345    261540.05    111.530938
2  Agência Tatuapé          2246    256697.00    114.290739

Bottom 3 agências (últimos 6 meses):
            nome_agencia  n_transacoes  valor_total  ticket_medio
7  Agência Florianópolis          1524    167771.51    110.086293
8        Agência Jardins          1121    157626.04    140.611989
9         Agência Recife           374    112926.01    301.941203


#07_EXPORTAÇÃO_ARQUIVOS

In [17]:
caminho_export = '/content/drive/My Drive/INDICIUM_DESAFIO/03_exports/'
# Exportando a tabela principal
df_final.to_csv(caminho_export + 'df_final_tratado_03.csv', index=False)

# <<< NOVAS LINHAS DE EXPORTAÇÃO ADICIONADAS AQUI >>>
# Exportando as novas tabelas de análise
analise_meses.to_csv(caminho_export + 'analise_meses_pares_impares.csv', index=False)
analise_dolar.to_csv(caminho_export + 'analise_dolar.csv', index=False)
dim_dates.to_csv(caminho_export + 'dim_dates.csv', index=False)
rank_ag.to_csv(caminho_export + 'rank_agencias_6m.csv', index=False)

print(f"\nArquivos exportados com sucesso para a pasta '{caminho_export}'!")

NameError: name 'dim_dates' is not defined